<a href="https://colab.research.google.com/github/ficle-fr/pix2pix_change_style/blob/colabs/colabs/train_tpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/ficle-fr/pix2pix_change_style/main/common.py
!wget https://raw.githubusercontent.com/ficle-fr/pix2pix_change_style/main/descriminator.py
!wget https://raw.githubusercontent.com/ficle-fr/pix2pix_change_style/main/generator.py
!wget https://raw.githubusercontent.com/ficle-fr/pix2pix_change_style/main/img_generator.py
!wget https://raw.githubusercontent.com/ficle-fr/pix2pix_change_style/main/train.py

#Currently from the colab branch
!wget https://raw.githubusercontent.com/ficle-fr/pix2pix_change_style/colabs/colabs/read_write_db.py


--2024-04-08 20:35:02--  https://raw.githubusercontent.com/ficle-fr/pix2pix_change_style/main/common.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2613 (2.6K) [text/plain]
Saving to: ‘common.py’

common.py           100%[===================>]   2.55K  --.-KB/s    in 0s      

2024-04-08 20:35:02 (41.5 MB/s) - ‘common.py’ saved [2613/2613]

--2024-04-08 20:35:02--  https://raw.githubusercontent.com/ficle-fr/pix2pix_change_style/main/descriminator.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1480 (1.4K) [text/plain]
Savin

In [2]:
!sudo apt install libcairo2-dev pkg-config python3-dev
!pip install pycairo

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
pkg-config is already the newest version (0.29.2-1ubuntu3).
python3-dev is already the newest version (3.10.6-1~22.04).
python3-dev set to manually installed.
The following additional packages will be installed:
  libblkid-dev libblkid1 libcairo-script-interpreter2 libffi-dev
  libglib2.0-dev libglib2.0-dev-bin libice-dev liblzo2-2 libmount-dev
  libmount1 libpixman-1-dev libselinux1-dev libsepol-dev libsm-dev
  libxcb-render0-dev libxcb-shm0-dev
Suggested packages:
  libcairo2-doc libgirepository1.0-dev libglib2.0-doc libgdk-pixbuf2.0-bin
  | libgdk-pixbuf2.0-dev libxml2-utils libice-doc cryptsetup-bin libsm-doc
The following NEW packages will be installed:
  libblkid-dev libcairo-script-interpreter2 libcairo2-dev libffi-dev
  libglib2.0-dev libglib2.0-dev-bin libice-dev liblzo2-2 libmount-dev
  libpixman-1-dev libselinux1-dev libsepol-dev libsm-dev libxcb-render0-dev
  libxcb-shm0-dev
The

In [9]:
import tensorflow as tf
import time

from generator import Generator, generator_loss
from descriminator import Discriminator, discriminator_loss

from img_generator import img_pair_gen1
from read_write_db import write, read

In [4]:
write("temp_records100.tfrecords", 100)

In [5]:
dataset = read("./temp_records100.tfrecords")
BATCH_SIZE = 5
dataset = dataset.batch(BATCH_SIZE)

In [6]:
print("Tensorflow version " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print("Total number of TPU cores:", tpu.get_tpu_system_metadata().num_cores)
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.TPUStrategy(tpu)

Tensorflow version 2.12.0
Total number of TPU cores: 0


In [26]:
@tf.function
def train_multiple_steps(generator, discriminator,
                         generator_optimizer, discriminator_optimizer,
                         steps, iterator):
    def step_fn(input, output):
        input_image, target = input
        with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
            gen_output = generator(input_image, training = True)

            disc_real_output = discriminator([input_image, target], training = True)
            disc_generated_output = discriminator([input_image, gen_output], training=True)

            gen_total_loss, gen_gan_loss, gen_l1_loss = generator_loss(disc_generated_output, gen_output, target)
            disc_loss = discriminator_loss(disc_real_output, disc_generated_output)

            generator_gradients = gen_tape.gradient(gen_total_loss,
                                                    generator.trainable_variables)
            discriminator_gradients = disc_tape.gradient(disc_loss,
                                                        discriminator.trainable_variables)

            generator_optimizer.apply_gradients(zip(generator_gradients,
                                                    generator.trainable_variables))
            discriminator_optimizer.apply_gradients(zip(discriminator_gradients,
                                                        discriminator.trainable_variables))

    #for input, output in dataset:
        #print("input:", input.shape)
        #print("output:", output.shape)
        #start = time.time()
        #tpu_strategy.run(step_fn, args=(input, output))
        #print(f'\nTraining time: {time.time()-start:.2f} sec\n')
    for _ in tf.range(steps):
    strategy.run(step_fn, args=(next(iterator),))


In [27]:
print(tpu_strategy.num_replicas_in_sync)
per_replica_batch_size = BATCH_SIZE // tpu_strategy.num_replicas_in_sync

train_dataset = tpu_strategy.experimental_distribute_datasets_from_function(
    lambda _: dataset)

with tpu_strategy.scope():
    generator = Generator([256, 256, 3], 3)
    discriminator = Discriminator([256, 256, 3])

    generator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
    discriminator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

train_multiple_steps(generator, discriminator,
                    generator_optimizer, discriminator_optimizer,
                    train_dataset)

8


/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/dataset_ops.py:467: UserWarning: To make it possible to preserve tf.data options across serialization boundaries, their implementation has moved to be part of the TensorFlow graph. As a consequence, the options value is in general no longer known at graph construction time. Invoking this method in graph mode retains the legacy behavior of the original implementation, but note that the returned value might not reflect the actual value of the options.
  warnings.warn("To make it possible to preserve tf.data options across "


ValueError: in user code:

    File "<ipython-input-22-0bc29ef21e40>", line 30, in train_multiple_steps  *
        tpu_strategy.run(step_fn, args=(input, output))

    ValueError: input tensor Tensor("while/OptionalGetValue:0", dtype=float32) to TPUStrategy.run() has unknown rank, which is not allowed
